## Minimal working example

Script creation for the LDclumping.ipynb

In [88]:
%save MWE_ldclump.sh -f

# Defining bash variables for the different paths,

output=output
bfile=~/MWE0612/genotypes.bed
bgenFile=`echo ~/MWE0612/imputed_genotypes_chr2?.bgen`
sampleFile=~/MWE0612/imputed_genotypes.sample
sumstatsFile=~/MWE0612/output/phenotypes_BMI.fastGWA.snp_stats.gz
unrelated_samples=~/MWE0612/unrelated_samples.txt
ld_sample_size=100
clump_field=P
clump_p1=0.05
clump_p2=1
clump_r2=0.2
clump_kb=1000
clump_annotate=OR
numThreads=20
job_size=1

#Running the workflow for BMI trait

sos dryrun ~/UKBB_GWAS_DEV/workflow/LD_Clumping.ipynb \
    --cwd $output \
    --bfile $bfile \
    --bgenFile $bgenFile \
    --sampleFile $sampleFile \
    --sumstatsFile $sumstatsFile \
    --unrelated_samples $unrelated_samples \
    --ld_sample_size $ld_sample_size \
    --clump_field $clump_field \
    --clump_p1 $clump_p1 \
    --clump_p2 $clump_p2 \
    --clump_r2 $clump_r2 \
    --clump_kb $clump_kb \
    --clump_annotate $clump_annotate \
    --numThreads $numThreads \
    --job_size $job_size
    -q localhost \
    -s build &> 062420-sos-LD-MWE.log

Cell content saved to MWE_ldclump.sh, use option -r to also execute the cell.

## MWE for REGENIE

In [38]:
cd
sos run ~/project/bioworkflows/GWAS/LMM.ipynb regenie\
    --cwd output \
    --bfile data/genotypes21_22.bed \
    --sampleFile data/imputed_genotypes.sample \
    --bgenFile data/imputed_genotypes_chr*.bgen \
    --phenoFile data/phenotypes.txt \
    --formatFile data/regenie_template.yml \
    --phenoCol BMI\
    --covarCol SEX \
    --qCovarCol AGE \
    --numThreads 8 \
    --bsize 1000 \
    --lowmem_prefix /home/dc2325/output \
    --trait \
    --minMAC 4 \
    --bgenMinMAF 0.05 \
    --bgenMinINFO 0.8 \
    --reverse_log_p \
    --p_filter 1\
    --container_option ~/scratch60/lmm.sif \
    --container_marp ~/scratch60/marp.sif

INFO: Running regenie_1: Run REGENIE step 1: fitting the null
INFO: regenie_1 is completed.
INFO: regenie_1 output:   output/phenotypes_BMI.regenie_pred.list
INFO: Running regenie_2: Run REGENIE step 2: association analysis
INFO: regenie_2 (index=0) is completed.
INFO: regenie_2 (index=1) is completed.
INFO: regenie_2 output:   output/imputed_genotypes_chr21_BMI.regenie.gz output/imputed_genotypes_chr22_BMI.regenie.gz in 2 groups
INFO: Running regenie_3: Merge results and log files
INFO: regenie_3 is completed.
INFO: regenie_3 output:   output/phenotypes_BMI.regenie.snp_stats.gz output/phenotypes_BMI.regenie.snp_counts.txt
INFO: Running regenie_4: Manhattan and QQ plots using `qqman`
INFO: regenie_4 is completed.
INFO: regenie_4 output:   output/phenotypes_BMI.regenie.manhattan.png output/phenotypes_BMI.regenie.qq.png... (5 items)
INFO: Running regenie_5: Generate analysis report: HTML file, and optionally PPTX file
/.singularity.d/actions/exec: exec: line 21: illegal option -o
ERROR: 

In [31]:
# Common variables
tpl_file=~/project/UKBB_GWAS_DEV/farnam.yml
bfile=~/data/genotypes.bed
sampleFile=~/data/imputed_genotypes.sample
bgenFile=~/data/imputed_genotypes_chr*.bgen
# LMM directories
lmm_dir_regenie=~/output
lmm_sos=~/project/bioworkflows/GWAS/LMM.ipynb
lmm_sbatch_regenie=../output/$(date +"%Y-%m-%d")_mwe_regenie.sbatch
phenoFile=~/data/phenotypes.txt
lowmem=~/output
## LMM variables 
phenoCol=BMI
covarCol=SEX
covarMaxLevels=10
qCovarCol=AGE
numThreads=8
bgenMinMAF=0.05
bgenMinINFO=0.8
minMAC=4
lmm_job_size=1
bsize=1000
ylim=0

In [32]:
lmm_args="""regenie
    --cwd $lmm_dir_regenie 
    --bfile $bfile 
    --sampleFile $sampleFile
    --bgenFile $bgenFile 
    --phenoFile $phenoFile  
    --phenoCol $phenoCol 
    --covarCol $covarCol 
    --qCovarCol $qCovarCol
    --bgenMinMAF $bgenMinMAF
    --bgenMinINFO $bgenMinINFO
    --bsize $bsize
    --minMAC $minMAC
    --lowmem $lowmem
    --numThreads $numThreads  
    --job_size $lmm_job_size
    --ylim $ylim
    --trait
"""

sos run ~/project/bioworkflows/GWAS/Get_Job_Script.ipynb farnam \
    --template-file $tpl_file \
    --workflow-file $lmm_sos \
    --to-script $lmm_sbatch_regenie \
    --args "$lmm_args"

INFO: Running farnam: Configuration for Yale `farnam` cluster
INFO: farnam is completed.
INFO: farnam output:   ../output/2020-08-10_mwe_regenie.sbatch
INFO: Workflow farnam (ID=280055fda8fd6626) is executed successfully with 1 completed step.


## MWE for BOLT-LMM

In [21]:
cd
sos run ~/project/bioworkflows/GWAS/LMM.ipynb boltlmm \
    --cwd output \
    --bfile data/genotypes.bed \
    --sampleFile data/imputed_genotypes.sample \
    --bgenFile data/imputed_genotypes_chr*.bgen \
    --phenoFile data/phenotypes.txt \
    --formatFile data/boltlmm_template.yml \
    --LDscoresFile ~/software/BOLT-LMM_v2.3.4/tables/LDSCORE.1000G_EUR.tab.gz \
    --geneticMapFile ~/software/BOLT-LMM_v2.3.4/tables/genetic_map_hg19_withX.txt.gz \
    --phenoCol BMI \
    --covarCol SEX \
    --covarMaxLevels 10 \
    --qCovarCol AGE \
    --numThreads 5 \
    --bgenMinMAF 0.001 \
    --bgenMinINFO 0.1 \
    --lmm-option \
    --p_filter 1 \
    --container_option ~/scratch60/lmm.sif

INFO: Running boltlmm_1: Run BOLT analysis
INFO: boltlmm_1 (index=0) is ignored due to saved signature
INFO: boltlmm_1 (index=1) is ignored due to saved signature
INFO: boltlmm_1 output:   output/cache/imputed_genotypes_chr21.phenotypes_BMI.boltlmm.snp_stats.gz output/cache/imputed_genotypes_chr22.phenotypes_BMI.boltlmm.snp_stats.gz in 2 groups
INFO: Running boltlmm_2: Merge results and log files
INFO: boltlmm_2 (index=0) is ignored due to saved signature
INFO: boltlmm_2 output:   output/phenotypes_BMI.boltlmm.snp_stats.gz output/phenotypes_BMI.boltlmm.snp_counts.txt
INFO: Running boltlmm_3: Manhattan and QQ plots using `qqman`
ERROR: boltlmm_3 (id=350b8d5153d17e93) returns an error.
ERROR: [boltlmm_3]: [0]: Executing script in Singularity returns an error (exitcode=1, stderr=output/phenotypes_BMI.boltlmm.manhattan.stderr, stdout=output/phenotypes_BMI.boltlmm.manhattan.stdout).
The script has been saved to /gpfs/ysm/home/dc2325/.sos/singularity_run_18938.R. To reproduce the error pleas

## MWE for FastGWA

In [19]:
sos run ~/project/bioworkflows/GWAS/LMM.ipynb fastGWA \
    --cwd output \
    --bfile data/genotypes.bed \
    --sampleFile data/imputed_genotypes.sample \
    --bgenFile data/imputed_genotypes_chr*.bgen \
    --phenoFile data/phenotypes.txt \
    --formatFile data/fastGWA_template.yml \
    --phenoCol BMI \
    --covarCol SEX \
    --qCovarCol AGE \
    --numThreads 1 \
    --bgenMinMAF 0.001 \
    --bgenMinINFO 0.1 \
    --parts 2 \
    --p_filter 1 \
    --container_option /SAY/dbgapstg/scratch/UKBiobank/lmm.sif \
    $JOB_OPT

INFO: Running fastGWA_1: fastGWA mixed model (based on the sparse GRM generated above)
INFO: Running gcta_1: Partition the GRM into 100 parts and allocate 8GB memory to each job
INFO: gcta_1 (index=0) is ignored due to saved signature
INFO: gcta_1 (index=1) is ignored due to saved signature
INFO: gcta_1 output:   output/cache/genotypes.part_2_1.grm.bin output/cache/genotypes.part_2_1.grm.N.bin... (6 items in 2 groups)
INFO: Running gcta_2: Merge all the parts together (Linux, Mac)
INFO: gcta_2 is completed.
INFO: gcta_2 output:   output/genotypes.grm.bin output/genotypes.grm.N.bin... (3 items)
INFO: Running gcta_3: Make a sparse GRM from the merged full-dense GRM
INFO: gcta_3 is completed.
INFO: gcta_3 output:   output/genotypes.grm.sp
INFO: fastGWA_1 (index=0) is ignored due to saved signature
INFO: fastGWA_1 (index=1) is ignored due to saved signature
INFO: fastGWA_1 output:   output/cache/imputed_genotypes_chr21.phenotypes.fastGWA.gz output/cache/imputed_genotypes_chr22.phenotypes.f